In [ ]:
import os
import pandas as pd
import cv2

# Set paths
images_dir = r"C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\images\train"
csv_path = r"C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\gt_train.csv"
labels_dir = r"C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\train"

# Create labels/train directory if it doesn't exist
os.makedirs(labels_dir, exist_ok=True)

# Read CSV, ensure image_id is treated as a string
df = pd.read_csv(csv_path, header=None, dtype={0: str})

# Map class names to YOLO class IDs
class_name_to_id = {
    'articulated_truck': 0,
    'bicycle': 1,
    'bus': 2,
    'car': 3,
    'motorcycle': 4,
    'motorized_vehicle': 5,
    'non-motorized_vehicle': 6,
    'pedestrian': 7,
    'pickup_truck': 8,
    'single_unit_truck': 9,
    'work_van': 10
}

# Group by image_id
grouped = df.groupby(0)

for image_id, group in grouped:
    img_path = os.path.join(images_dir, image_id + ".jpg")
    label_path = os.path.join(labels_dir, image_id + ".txt")
    
    if not os.path.exists(img_path):
        continue  # Skip if image file doesn't exist
    
    img = cv2.imread(img_path)
    if img is None:
        continue  # Skip if image file is corrupted or unreadable
    
    h, w = img.shape[:2]
    
    lines = []
    for _, row in group.iterrows():
        class_name = row[1].lower()
        x1, y1, x2, y2 = row[2], row[3], row[4], row[5]
        
        x_center = (x1 + x2) / 2 / w
        y_center = (y1 + y2) / 2 / h
        bbox_width = (x2 - x1) / w
        bbox_height = (y2 - y1) / h
        
        if class_name not in class_name_to_id:
            continue
        
        class_id = class_name_to_id[class_name]
        
        line = f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}"
        lines.append(line)
    
    # Write label file only if there are valid objects
    if lines:
        with open(label_path, 'w') as f:
            f.write('\n'.join(lines))

print("Done: All labels have been generated in labels/train/")


✅ 完成：所有標籤已生成到 labels/train/


In [ ]:
import os
import random
import shutil

# Original train folder paths
images_train_dir = r"C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\images\train"
labels_train_dir = r"C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\train"

# New validation folder paths
images_val_dir = r"C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\images\val"
labels_val_dir = r"C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\val"

# Create val directories if they don't exist
os.makedirs(images_val_dir, exist_ok=True)
os.makedirs(labels_val_dir, exist_ok=True)

# List all image files (.jpg only)
image_files = [f for f in os.listdir(images_train_dir) if f.endswith('.jpg')]

# Shuffle randomly
random.shuffle(image_files)

# Split ratio
val_ratio = 0.1
val_count = int(len(image_files) * val_ratio)

val_images = image_files[:val_count]     # First 10% goes to validation
train_images = image_files[val_count:]   # Remaining 90% stays in training

# Move validation data
for img_file in val_images:
    img_src = os.path.join(images_train_dir, img_file)
    img_dst = os.path.join(images_val_dir, img_file)

    label_file = img_file.replace('.jpg', '.txt')
    label_src = os.path.join(labels_train_dir, label_file)
    label_dst = os.path.join(labels_val_dir, label_file)

    # Move image
    shutil.move(img_src, img_dst)

    # Move label if it exists
    if os.path.exists(label_src):
        shutil.move(label_src, label_dst)

print(f"Split complete!")
print(f"Training set size (images/train): {len(train_images)} images")
print(f"Validation set size (images/val): {len(val_images)} images")


✅ 完成切分！
訓練集數量（images/train）：99000張
驗證集數量（images/val）：11000張


In [ ]:
import torch
import ultralytics
import cv2
import numpy as np
import matplotlib.pyplot as plt

print("PyTorch version:", torch.__version__)
print("Is CUDA available:", torch.cuda.is_available())
print("YOLO version:", ultralytics.__version__)
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CUDA not available")


PyTorch 版本: 2.5.1+cu121
CUDA 是否可用: True
YOLO 版本: 8.3.96
使用的 GPU 名稱： NVIDIA GeForce RTX 4060


In [ ]:
import os
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from ultralytics import YOLO

# 1. Automatically generate an experiment name with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
exp_name = f'yolov8_testlog_{timestamp}'
logdir = os.path.join('runs/train', exp_name)

# 2. Force-initialize TensorBoard writer to ensure log directory is created
os.makedirs(logdir, exist_ok=True)
writer = SummaryWriter(log_dir=logdir)
writer.add_scalar('init_scalar_force_log', 0, 0)  # Dummy value to force .tfevents file
writer.close()

# 3. Load the model (can be yolov8s.pt, yolov8m.pt, or a custom checkpoint)
model = YOLO(r"C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250428_1507512\weights\last.pt")

# 4. Start training using custom dataset
# If the dataset file is not found, Ultralytics will attempt to auto-download coco128
model.train(
    data=r"C:\Users\jj111\Desktop\new_train\data.yaml",  # Path to your dataset YAML
    epochs=100,
    imgsz=640,
    batch=32,
    workers=8,
    amp=True,                     # Enable Automatic Mixed Precision
    device=0,                     # GPU index (0 for first GPU)
    project="runs/train",
    name=exp_name,
    verbose=True,
    save_period=5                # Save every 5 epochs
)

print(f"\n Training complete! You can view the results with TensorBoard:")
print(f"tensorboard --logdir runs/train/{exp_name}")


New https://pypi.org/project/ultralytics/8.3.122 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.96  Python-3.10.1 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250428_1507512\weights\last.pt, data=C:\Users\jj111\Desktop\new_train\data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=8, project=runs/train, name=yolov8_testlog_20250430_1637332, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, strea

train: Scanning C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\train.cache... 99000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 99000/99000 [00:00<?, ?it/s]
val: Scanning C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\val.cache... 11000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11000/11000 [00:00<?, ?it/s]


Plotting labels to runs\train\yolov8_testlog_20250430_1637332\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\train\yolov8_testlog_20250430_1637332
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      7.22G     0.8196     0.4912     0.9143        130        640: 100%|██████████| 3094/3094 [1:47:05<00:00,  2.08s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [02:15<00:00,  1.27it/s]


                   all      11000      35047      0.788       0.79      0.828      0.632

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all      11000      35047      0.792      0.789      0.831      0.636



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     40/100      7.13G     0.8685     0.5701     0.9312        134        640: 100%|██████████| 3094/3094 [1:47:01<00:00,  2.08s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [02:58<00:00,  1.04s/it]


                   all      11000      35047      0.788      0.793      0.831      0.636

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     41/100      7.14G     0.8678     0.5686     0.9309        160        640:  92%|█████████▏| 2841/3094 [1:44:12<10:00,  2.38s/it]

In [ ]:
from ultralytics import YOLO

model = YOLO("C:/Users/jj111/Desktop/new_train/runs/train/yolov8_testlog_20250430_1637332/weights/last.pt")

model.train(
    resume=True,
)


New https://pypi.org/project/ultralytics/8.3.124 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.96  Python-3.10.1 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt, data=C:\Users\jj111\Desktop\new_train\data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=8, project=runs/train, name=yolov8_testlog_20250430_1637332, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou

train: Scanning C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\train.cache... 99000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 99000/99000 [00:00<?, ?it/s]
val: Scanning C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\val.cache... 11000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11000/11000 [00:00<?, ?it/s]


Plotting labels to runs\train\yolov8_testlog_20250430_1637332\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Resuming training C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt from epoch 41 to 100 total epochs
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\train\yolov8_testlog_20250430_1637332
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     41/100      7.03G     0.8594     0.5579     0.9271        130        640: 100%|██████████| 3094/3094 [21:55<00:00,  2.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [01:24<00:00,  2.05it/s]

                   all      11000      35047      0.788      0.793      0.831      0.636



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     42/100      7.06G     0.8643     0.5645     0.9272        124        640: 100%|██████████| 3094/3094 [44:34<00:00,  1.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [01:14<00:00,  2.31it/s]

                   all      11000      35047      0.791       0.79      0.832      0.637



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     43/100      7.06G     0.8636     0.5628     0.9296        182        640:  48%|████▊     | 1487/3094 [23:42<21:02,  1.27it/s]

In [ ]:
from ultralytics import YOLO


model = YOLO("C:/Users/jj111/Desktop/new_train/runs/train/yolov8_testlog_20250430_1637332/weights/last.pt")

model.train(
    resume=True,           
)


New https://pypi.org/project/ultralytics/8.3.124 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.96  Python-3.10.1 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt, data=C:\Users\jj111\Desktop\new_train\data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=8, project=runs/train, name=yolov8_testlog_20250430_1637332, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou

train: Scanning C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\train.cache... 99000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 99000/99000 [00:00<?, ?it/s]
val: Scanning C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\val.cache... 11000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11000/11000 [00:00<?, ?it/s]


Plotting labels to runs\train\yolov8_testlog_20250430_1637332\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Resuming training C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt from epoch 43 to 100 total epochs
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\train\yolov8_testlog_20250430_1637332
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     43/100      6.98G     0.8518      0.547     0.9245        130        640: 100%|██████████| 3094/3094 [47:55<00:00,  1.08it/s]  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [01:39<00:00,  1.73it/s]


                   all      11000      35047       0.79      0.792      0.832      0.637

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     44/100      6.98G     0.8614     0.5609     0.9292        124        640: 100%|██████████| 3094/3094 [40:50<00:00,  1.26it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [01:33<00:00,  1.85it/s]

                   all      11000      35047      0.786      0.796      0.832      0.637



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     45/100      6.98G     0.8626     0.5612     0.9301        154        640:  46%|████▌     | 1409/3094 [19:11<28:37,  1.02s/it]

In [ ]:
from ultralytics import YOLO


model = YOLO("C:/Users/jj111/Desktop/new_train/runs/train/yolov8_testlog_20250430_1637332/weights/last.pt")

model.train(
    resume=True,           
)

New https://pypi.org/project/ultralytics/8.3.124 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.96  Python-3.10.1 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt, data=C:\Users\jj111\Desktop\new_train\data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=8, project=runs/train, name=yolov8_testlog_20250430_1637332, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou

train: Scanning C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\train.cache... 99000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 99000/99000 [00:00<?, ?it/s]
val: Scanning C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\val.cache... 11000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11000/11000 [00:00<?, ?it/s]


Plotting labels to runs\train\yolov8_testlog_20250430_1637332\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Resuming training C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt from epoch 45 to 100 total epochs
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\train\yolov8_testlog_20250430_1637332
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     45/100      6.98G     0.8442     0.5361     0.9218        130        640: 100%|██████████| 3094/3094 [45:11<00:00,  1.14it/s] 
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [01:21<00:00,  2.10it/s]

                   all      11000      35047      0.784      0.795      0.832      0.637


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     53/100      6.98G     0.8568     0.5528     0.9252        122        640: 100%|██████████| 3094/3094 [36:33<00:00,  1.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [00:54<00:00,  3.16it/s]

                   all      11000      35047      0.796      0.793      0.834      0.639



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     54/100      6.98G     0.8542     0.5528     0.9261        199        640:  13%|█▎        | 411/3094 [04:49<31:23,  1.42it/s]

In [1]:
from ultralytics import YOLO


model = YOLO("C:/Users/jj111/Desktop/new_train/runs/train/yolov8_testlog_20250430_1637332/weights/last.pt")

model.train(
    resume=True,           
)

New https://pypi.org/project/ultralytics/8.3.124 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.96  Python-3.10.1 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt, data=C:\Users\jj111\Desktop\new_train\data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=8, project=runs/train, name=yolov8_testlog_20250430_1637332, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou

train: Scanning C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\train.cache... 99000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 99000/99000 [00:00<?, ?it/s]
val: Scanning C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\val.cache... 11000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11000/11000 [00:00<?, ?it/s]


Plotting labels to runs\train\yolov8_testlog_20250430_1637332\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Resuming training C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt from epoch 54 to 100 total epochs
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\train\yolov8_testlog_20250430_1637332
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     54/100      6.98G      0.834     0.5209     0.9169        130        640: 100%|██████████| 3094/3094 [1:20:55<00:00,  1.57s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [02:32<00:00,  1.13it/s]

                   all      11000      35047      0.794      0.797      0.835      0.639


...

     60/100      6.98G     0.8512     0.5416     0.9239        137        640: 100%|██████████| 3094/3094 [1:40:05<00:00,  1.94s/it]  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [05:10<00:00,  1.81s/it]

                   all      11000      35047      0.793      0.795      0.835       0.64



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     61/100      6.98G      0.804     0.5639     0.9263        160        640:   0%|          | 1/3094 [00:05<4:21:06,  5.07s/it]


KeyboardInterrupt: 

In [4]:
from ultralytics import YOLO


model = YOLO("C:/Users/jj111/Desktop/new_train/runs/train/yolov8_testlog_20250430_1637332/weights/last.pt")

model.train(
    resume=True,           
)

New https://pypi.org/project/ultralytics/8.3.126 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.96  Python-3.10.1 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt, data=C:\Users\jj111\Desktop\new_train\data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=8, project=runs/train, name=yolov8_testlog_20250430_1637332, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou

train: Scanning C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\train.cache... 99000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 99000/99000 [00:00<?, ?it/s]
val: Scanning C:\Users\jj111\Desktop\new_train\MIO-TCD-Localization\labels\val.cache... 11000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11000/11000 [00:00<?, ?it/s]


Plotting labels to runs\train\yolov8_testlog_20250430_1637332\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Resuming training C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\last.pt from epoch 61 to 100 total epochs
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\train\yolov8_testlog_20250430_1637332
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     61/100      7.27G      0.822     0.5029      0.914        130        640: 100%|██████████| 3094/3094 [1:30:46<00:00,  1.76s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [02:27<00:00,  1.17it/s]

                   all      11000      35047      0.792      0.795      0.835       0.64


...

     80/100       6.9G     0.8313     0.5098     0.9156        104        640: 100%|██████████| 3094/3094 [46:12<00:00,  1.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [01:28<00:00,  1.95it/s]

                   all      11000      35047      0.805      0.793      0.839      0.645



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     81/100       6.9G     0.8264     0.5063     0.9124        208        640:   0%|          | 10/3094 [00:09<47:50,  1.07it/s]


KeyboardInterrupt: 